In [5]:
from datetime import datetime

import geopandas
import pandas as pd
import requests
import numpy as np

from helpers import *

In [6]:
data = geopandas.read_file("EFSM20_CF_PLD.geojson")
data = data[data.idsource.str.startswith("NAF")]  # north anatolian fault

In [7]:
data

,idfs,idds,idsource,strikemin,strikeavg,strikemax,dipmin,dipavg,dipmax,rakemin,...,mwmaxp02,mwmaxp05,mwmaxavg,mwmaxp95,mwmaxp98,m0rmin,m0rmax,m0ramean,m0rgmean,geometry
419,TRCF045,#16,NAF01,242.0,262.0,299.0,59.0,74.0,90.0,-179.0,...,7.69,7.74,7.96,8.27,8.37,18.6423,18.9648,18.8058,18.7872,"POLYGON ((24.94460 40.21030, 24.94600 40.21100..."
876,TRCF000,#16,NAF03,90.0,108.0,127.0,85.0,87.0,90.0,177.0,...,7.64,7.69,7.91,8.22,8.32,18.7651,18.8582,18.8138,18.8114,"POLYGON ((41.12690 39.31120, 41.12670 39.31120..."
877,TRCF001,#16,NAF04,239.0,244.0,254.0,85.0,87.0,90.0,-178.0,...,7.21,7.26,7.48,7.79,7.89,17.0094,17.4882,17.3110,17.2485,"POLYGON ((34.08910 39.93340, 34.08930 39.93350..."
1120,TRCF046,#16,NAF01,264.0,273.0,289.0,59.0,74.0,90.0,-179.0,...,6.66,6.71,6.93,7.24,7.34,17.3966,17.5093,17.4372,17.4366,"POLYGON ((30.93380 40.79320, 30.93400 40.79330..."
1121,TRCF047,#16,NAF02,250.0,268.0,293.0,74.0,82.0,90.0,-179.0,...,7.64,7.69,7.91,8.22,8.32,18.7887,18.8250,18.8026,18.8025,"POLYGON ((31.25320 40.63830, 31.25420 40.63860..."
1122,TRCF048,#16,NAF02,256.0,274.0,286.0,74.0,82.0,90.0,-179.0,...,6.83,6.88,7.10,7.41,7.51,17.6773,17.7323,17.7009,17.7004,"POLYGON ((30.41700 40.64530, 30.42450 40.66740..."


In [8]:
coordinates = []
for i in range(data.shape[0]):
    coordinates.append(data["geometry"].iloc[i].bounds + (data['idsource'].iloc[i],))

In [9]:
coordinates

[(24.9446, 40.1727, 30.9414, 40.9139, 'NAF01'),
 (35.9319, 39.3112, 41.1301, 40.5724, 'NAF03'),
 (34.0891, 39.9273, 35.9319, 40.578, 'NAF04'),
 (30.9338, 40.7325, 31.5057, 40.8154, 'NAF01'),
 (31.2532, 40.617, 36.6608, 41.1407, 'NAF02'),
 (30.417, 40.5743, 31.2635, 40.6674, 'NAF02')]

In [10]:
datas = []
for c in coordinates:
    # dataurl = f"https://deprem.afad.gov.tr/apiv2/event/filter?limit=1000000&minlat={c[1]}&maxlat={c[3]}&minlon={c[0]}&maxlon={c[2]}&format=json&start=1900-01-01%2000%3A00%3A00&end={datetime.today().isoformat()}&orderby=timedesc"
    dataurl = f"https://deprem.afad.gov.tr/apiv2/event/filter?limit=1000000&minlat={c[1]}&maxlat={c[3]}&minlon={c[0]}&maxlon={c[2]}&format=json&start=2014-11-27&end={datetime.today().isoformat()}&orderby=timedesc&minmag=3.5"
    data = requests.get(dataurl)
    assert data.status_code == 200
    data = data.json()
    data = pd.DataFrame(data)
    data = data.assign(idsource=c[4])
    datas.append(data)

datas = pd.concat(datas, ignore_index=True)


In [11]:
datas = datas.astype({
    'rms': np.float64,
    'eventID': str,
    'location': str,
    'latitude': np.float64,
    'longitude': np.float64,
    'depth': np.float64,
    'type': str,
    'magnitude': np.float64,
    'country': str,
    'province': str,
    'district': str,
    'neighborhood': str,
    'date': np.datetime64
})

In [12]:
datas

,rms,eventID,location,latitude,longitude,depth,type,magnitude,country,province,district,neighborhood,date,isEventUpdate,lastUpdateDate,idsource
0,0.30,538260,Biga (Çanakkale),40.3640,27.1100,12.12,MW,4.3,Türkiye,Çanakkale,Biga,Bekirli,2022-12-13 03:21:16.000,False,None,NAF01
1,0.37,521277,Marmara Denizi - [06.73 km] Şarköy (Tekirdağ),40.7186,27.4040,10.42,Mw,4.1,Türkiye,Tekirdağ,Şarköy,None,2022-02-20 20:20:11.400,False,None,NAF01
2,0.37,520422,Marmara Denizi - Gemlik Körfezi - [04.61 km] A...,40.4266,28.9930,7.04,Mw,3.9,Türkiye,Yalova,Armutlu,None,2022-02-06 12:07:53.260,False,None,NAF01
3,0.50,519660,Yenişehir (Bursa),40.2816,29.6661,7.74,Mw,3.7,Türkiye,Bursa,Yenişehir,Kurtuluş,2022-01-22 11:13:45.170,False,None,NAF01
4,0.36,501787,Marmara Denizi - Erdek Körfezi - [04.19 km] Bi...,40.3460,27.3946,10.24,Mw,4.0,Türkiye,Çanakkale,Biga,None,2021-05-05 06:42:11.830,False,None,NAF01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,0.34,335853,Merkez (Bolu),40.8536,31.7273,13.42,Mw,3.6,Türkiye,Bolu,Merkez,Tetemeçele,2016-05-01 02:55:31.700,False,None,NAF02
221,0.32,332879,Merkez (Kastamonu),41.0953,33.8253,3.05,Mw,3.7,Türkiye,Kastamonu,Merkez,Ahlat,2016-04-10 15:06:52.830,False,None,NAF02
222,0.34,323315,Çerkeş (Çankırı),40.6931,32.7601,10.02,Mw,3.7,Türkiye,Çankırı,Çerkeş,Yoncalı,2016-01-18 05:46:33.590,False,None,NAF02
223,0.36,302627,Merkez (Düzce),40.8628,31.2816,13.41,Mw,3.8,Türkiye,Düzce,Merkez,Asar,2015-07-07 05:08:30.000,False,None,NAF02


In [13]:
# Check updated events
datas.eventID.value_counts()

538260    1
475673    1
372839    1
371393    1
371389    1
         ..
535800    1
533252    1
533237    1
533235    1
296496    1
Name: eventID, Length: 225, dtype: int64

In [14]:
# Sort by time, take the last
datas.sort_values(by="date", inplace=True)
datas.drop_duplicates(subset="eventID", keep="last", inplace=True)

In [15]:
# We need to convert all magnitudes to same types
datas.type.value_counts()

Mw    193
ML     19
MW     13
Name: type, dtype: int64

In [16]:
# Make them lowercase 
datas.type = datas.type.str.lower()
datas.type.value_counts()

mw    206
ml     19
Name: type, dtype: int64

In [72]:
# I don't have a conversion formula for mwp
# Omit that obs
datas = datas[datas.type != 'mwp']
datas.shape

(41227, 16)

In [17]:
# Apply conversation formulas
# Source: https://aj.tubitak.gov.tr/earth/issues/yer-16-25-4/yer-25-4-2-1511-7.pdf
datas['magnitude'] = datas[['magnitude', 'type']].apply(convertif, axis=1)

In [18]:
# All magnitudes are in Mw now
# Drop columns
cols = ['type', 'neighborhood', 'isEventUpdate', 'lastUpdateDate']
datas.drop(labels=cols, axis=1, inplace=True)
datas

,rms,eventID,location,latitude,longitude,depth,magnitude,country,province,district,date,idsource
75,0.23,282592,Ege Denizi - [32.70 km] Gökçeada (Çanakkale),40.1985,25.3365,27.96,3.7,Türkiye,Çanakkale,Gökçeada,2014-11-27 12:19:00.880,NAF01
74,0.48,282992,Ege Denizi - [47.31 km] Gökçeada (Çanakkale),40.2465,25.1758,27.03,4.3,Türkiye,Çanakkale,Gökçeada,2014-12-02 20:15:17.080,NAF01
73,0.30,288850,Ege Denizi - [11.58 km] Gökçeada (Çanakkale),40.3083,26.0295,14.63,4.1,Türkiye,Çanakkale,Gökçeada,2015-02-02 04:41:04.420,NAF01
162,0.78,289348,Pülümür (Tunceli),39.4301,40.1351,13.01,4.3,Türkiye,Tunceli,Pülümür,2015-02-09 22:52:49.610,NAF03
161,0.39,291244,Aşkale (Erzurum),39.9063,40.8440,1.58,3.8,Türkiye,Erzurum,Aşkale,2015-02-26 10:06:20.350,NAF03
...,...,...,...,...,...,...,...,...,...,...,...,...
179,0.50,537527,Gümüşova (Düzce),40.8080,30.9460,14.20,4.1,Türkiye,Düzce,Gümüşova,2022-12-03 04:59:20.000,NAF01
77,0.42,538044,Refahiye (Erzincan),39.8540,38.6840,7.05,3.6,Türkiye,Erzincan,Refahiye,2022-12-10 10:29:26.000,NAF03
0,0.30,538260,Biga (Çanakkale),40.3640,27.1100,12.12,4.3,Türkiye,Çanakkale,Biga,2022-12-13 03:21:16.000,NAF01
76,0.49,538692,Aşkale (Erzurum),39.8590,40.5370,10.21,3.9,Türkiye,Erzurum,Aşkale,2022-12-18 08:57:49.000,NAF03


In [19]:
datas.to_csv("dataset_2.csv", index=False)